In [1]:
!pip install requests
!pip install beautifulsoup4

In [2]:
import requests
from bs4 import BeautifulSoup

Парсинг ссылок с первой страницы https://lifehacker.ru/topics/technology

Так как все ссылки в рубрике технологии имеют формат вида lifehacker.ru/*some_news*, примем за base_url - lifehacker.ru, *some_news* - уникальная ссылка на материал

In [3]:
base_url = 'https://lifehacker.ru'
response = requests.get('https://lifehacker.ru/topics/technology').text
soup = BeautifulSoup(response, 'lxml')

В среде разработки https://lifehacker.ru/topics/technology ссылки на материалы содержатся в 'href' под идентификатором 'a' и классом 'lh-small-article-card__link'.

In [5]:
raw_items = soup.find_all('a', class_='lh-small-article-card__link')
links = [item.get('href') for item in raw_items]
links

['/reklama-v-puske-teper-u-vsex/',
 '/funkcii-windows-net-na-mac/',
 '/apple-provedet-prezentaciyu-7-maya/',
 '/2gis-vernulos-v-google-play/',
 '/usilitel-signala-wi-fi/?erid=25H8d7vbP8SRTvJ4Q27doN',
 '/svyaz-s-voyadzherom-1-vosstanovili/',
 '/iphone-16-mozhet-lishitsya-fizicheskix-knopok/',
 '/umnye-chasy-s-ekg/?erid=25H8d7vbP8SRTvJ4Q27doN',
 '/vzlom-telegram-ot-sluzhby-podderzhki/',
 '/apple-pay-ne-zarabotaet-v-rossii/',
 '/besplatnye-graficheskie-redaktory/',
 '/emulyator-provenance-poyavitsya-v-app-store/',
 '/otparivatel/?erid=25H8d7vbP8SRTvJ4Q27doN',
 '/predstavlen-huawei-matebook-x-pro/',
 '/napominaniya-google-keep-nakonec-to-poyavyatsya-v-zadachax-i-kalendare/',
 '/predstavlena-ii-klipsa-pendant/',
 '/dostavka-iz-kosmosa-sierra-space/',
 '/predstavleny-naushniki-nothing-ear/',
 '/missiya-dragonfly-na-sputnik-titan/',
 '/huawei-pura-70-anons/',
 '/pravilo-zashhity-dannyx-3-2-1/',
 '/predstavleny-audioochki-yinyue-fan/',
 '/dlya-samyx-krutyx-snimkov-kanal-tech-spurt-vybral-12-lu

Парсинг ссылок со второй по десятую страницы https://lifehacker.ru/topics/technology

In [6]:
parsed_urls = []

In [7]:
for page_num in range(2, 11):
  url = f'{base_url}/topics/technology/?page={page_num}'
  response = requests.get(url)
  soup = BeautifulSoup(response.text, 'lxml')
  raw_items = soup.find_all('a', class_='lh-small-article-card__link')
  parsed_urls += [item.get('href') for item in raw_items]

In [8]:
parsed_urls

['/ii-v-ios-18-budet-lokalnym/',
 '/gravitacionnye-anomalii-okolo-solnca/',
 '/prekraschenie-podderzhki-office-2016-i-office-2019/',
 '/sliv-anonsov-ustrojstv-samsung/',
 '/problemy-dostavki-marsianskoj-porody/',
 '/fajlovye-menedzhery-dlya-android-tv/',
 '/sony-ps5-xarakteristiki/',
 '/smartfony-samsung-v-1-kvartale-2024-goda/',
 '/razdrazhayushhie-funkcii-iphone/',
 '/yaponiya-postroit-lunnyj-avtomobil/',
 '/xiaomi-umnoe-kolco-black-shark/',
 '/doverie-iskusstvennomu-intellektu/',
 '/v-telegram-sdelat-stiker/',
 '/byudzhetnye-umnye-chasy/?erid=25H8d7vbP8SRTvJ4Q27doN',
 '/predstavlen-ssd-nakopitel-ultrastar-transporter/',
 '/mac-pro-s-m4-vyjdet-v-2025/',
 '/dostup-k-zakladkam-v-chrome-izmenilsya/',
 '/samsung-rasshirit-rasprostranenie-galaxy-ai/',
 '/acloset/',
 '/v-puske-poyavitsya-reklama/',
 '/bystraya-zaryadka-v-smartfone/?erid=25H8d7vbP8SRTvJ4Q27doN',
 '/google-zapuskaet-chrome-enterprise-premium/',
 '/na-youtube-poyavitsya-ogranichenie-kommentariev/',
 '/predstavleny-xiaomi-open

In [9]:
len(parsed_urls)

270

In [10]:
len(links)

30

Соединим два наших массива ссылок, чтобы получить общий массив с 1 по 10 страницы включительно

In [11]:
total_urls = links + parsed_urls
len(total_urls)

300

In [12]:
from tqdm import tqdm

Полный html код каждого материала - f'{base_url}{url}'

Заголовок из анализа страницы одного материала - "лежит" под идентификатором 'h1' и классом 'article-card__title'.

Подзаголовок - под идентификатором 'div' и классом 'article-card__subtitle'.

Текст статьи - в классе - контейнере 'single-article__post-content single-article__content-container'

In [14]:
result = []

for url in tqdm(total_urls):
  articles = {}

  articles_url = f'{base_url}{url}'
  response = requests.get(articles_url)
  soup = BeautifulSoup(response.text, 'lxml')

  articles['title'] = soup.find('h1', class_='article-card__title').text
  articles['subtitle'] = soup.find('div', class_='article-card__subtitle').text
  articles['text'] = soup.find(class_='single-article__post-content single-article__content-container').text

  result.append(articles)

100%|██████████| 300/300 [06:58<00:00,  1.40s/it]


In [16]:
import pandas as pd

In [17]:
pd.set_option('display.max_colwidth', 400)

Создаем датафрейм из полученных данных

In [18]:
data = pd.DataFrame(result)
data.head()

,title,subtitle,text
0,Реклама в меню «Пуск» Windows 11 теперь будет отображаться у всех,Её можно легко отключить.,"Microsoft приступила к распространению обновления KB5036980 для Windows 11, которое включает в том числе рекламу прямо в меню «Пуск». Теперь она будет отображаться у всех пользователей.Интересно, что корпорация начала тестировать эту «функцию» всего две недели назад, и тот факт, что она за такой короткий срок прошла путь от бета до выпуска, немало удивляет. К слову, официально её называют «рек..."
1,"6 функций Windows, которых не хватает в macOS","Найти альтернативу на Mac можно, но только с костылями.","При сравнениях Windows и macOS обычно выделяются функции, которые есть на Mac и не имеют альтернативы в экосистеме Microsoft. Но в Windows тоже есть фишки, которых в macOS не найти — по крайней мере, без установки стороннего ПО или ухищрений, неочевидных обычному пользователю. Я расскажу именно о таких функциях.1. Многооконный режимИллюстрация: Дарья Громова / ЛайфхакерВполне ожидаемо от систе..."
2,Apple анонсировала «специальное мероприятие» 7 мая. Ждём новые iPad?,Могут также представить Apple Pencil и клавиатуру Magic Keyboard следующего поколения.,"Apple начала приглашать СМИ на «специальное мероприятие», которое пройдёт в виртуальном формате 7 мая. На пригласительной открытке изображён Apple Pencil. Это совершенно ясно даёт понять, что в центре внимания будут новые планшеты, пишет The Verge.Ранее различные источники сообщали, что iPad Pro получит некоторые из наиболее значимых улучшений со времён капитального ремонта в 2021 году, когда ..."
3,Спустя почти год «2ГИС» вернули в Google Play. Перейти к приложению можно по ссылке,Сразу с четырьмя новыми функциями.,"Бесплатный картографический сервис «2ГИС» снова доступен в Google Play. Его не было в магазине Android почти год — с мая 2023 года.Чтобы загрузить приложение на устройство, достаточно просто найти его через поиск в маркетплейсе, а затем нажать кнопку «Установить». Если стандартный способ не работает, то скачать его можно по этой ссылке. Мы очень признательны нашим пользователям: во время отсу..."
4,Зачем нужен усилитель сигнала Wi-Fi и как его выбрать,С помощью такого устройства можно просто и быстро расширить зону действия сети. Но есть нюансы.,"Как работает усилитель сигнала Wi‑FiУсилитель сигнала принимает сигнал от роутера и тут же передаёт его дальше. При этом зона охвата сети расширяется так, будто в точке установки усилителя стоит ещё один роутер. Но, вопреки своему названию, устройство не может усилить слабый сигнал: какой приняло, такой и повторило (поэтому устройство ещё называют повторителем, или репитером).Изображение: TP‑L..."
